In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import sys
import json
from datetime import datetime
from tqdm import tqdm
import pytz
%matplotlib inline

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Training and Testing directories
training_dir = os.path.join("Datasets", "Training")
testing_dir = os.path.join("Datasets", "Testing")
if not os.path.isdir(training_dir):
    raise Exception("ERROR: training dataset not found")
if not os.path.isdir(testing_dir):
    raise Exception("ERROR: testing dataset not found")

In [3]:
# iterate over all hashtag files 
for root, dirs, files in os.walk(training_dir, topdown=False):
    for file in files:
        print(os.path.join(root, file))

Datasets\Training\tweets_#gohawks.txt
Datasets\Training\tweets_#gopatriots.txt
Datasets\Training\tweets_#nfl.txt
Datasets\Training\tweets_#patriots.txt
Datasets\Training\tweets_#sb49.txt
Datasets\Training\tweets_#superbowl.txt


In [4]:
# Initialize dictionaries and Unix times for Feb 1, 8 am and Feb 1, 8 pm. 
# Dictionary keys: hashtag.
# Dictionary values: [time of tweet (Unix), number of retweets for tweet, number of followers for tweeter]
# Each row in dictionary value is an individual tweet.

hashtag_dict_before = {}
hashtag_dict_during = {}
hashtag_dict_after = {}
start_unix_time = 1422806400 # 8 am, Feb 1, PST
end_unix_time = 1422849600 # 8 pm, Feb 1, PST
pst_tz = pytz.timezone('America/Los_Angeles')


In [5]:
for root, dirs, files in os.walk(training_dir, topdown=False):
    for file in files:
        filename = os.path.splitext(file)[0].replace('tweets_#', '')
        print('Parsing {}...'.format(filename))
        
        hashtag_dict_before[filename] = []
        hashtag_dict_during[filename] = []
        hashtag_dict_after[filename] = []
        
        # open the file and read all lines:
        with open(os.path.join(root, file), "r", encoding="utf-8") as hashtag:
            # read line-by-line
            for line in hashtag:
                json_obj = json.loads(line)
                
                # get desired statistics
                citation_date = json_obj['citation_date'] # Unix time
                num_retweets = json_obj['metrics']['citations']['total'] # Number of retweets for this tweet
                num_followers = json_obj['author']['followers'] # Number of followers for tweeter
                
                # Check when tweet was made and add it to corresponding dictionary
                if citation_date < start_unix_time:
                    hashtag_dict_before[filename].append([citation_date, num_retweets, num_followers])
                elif citation_date > end_unix_time:
                    hashtag_dict_after[filename].append([citation_date, num_retweets, num_followers])
                else:
                    hashtag_dict_during[filename].append([citation_date, num_retweets, num_followers])
    print('done')

Parsing gohawks...
Parsing gopatriots...
Parsing nfl...
Parsing patriots...
Parsing sb49...
Parsing superbowl...
done


In [6]:
# Explicitly list hashtags. 
# Convert each value in dictionary to numpy arrays.

hashtags = ['gohawks', 'gopatriots', 'nfl', 'patriots', 'sb49', 'superbowl']

for key in hashtags:
    hashtag_dict_before[key] = np.array(hashtag_dict_before[key])
    hashtag_dict_during[key] = np.array(hashtag_dict_during[key])
    hashtag_dict_after[key] = np.array(hashtag_dict_after[key])

In [7]:
# Find how many time windows there are

ftt = int(np.min([np.min(hashtag_dict_before[key][:,0]) for key in hashtags])) # first tweet time
ltt = int(np.max([np.max(hashtag_dict_after[key][:,0]) for key in hashtags])) # last tweet time

num_windows_before = int(np.max([((start_unix_time - ftt) // 3600) + 1 for key in hashtags]))
num_windows_during = int(np.max([((end_unix_time - start_unix_time) // 3600 * 12) for key in hashtags]))
num_windows_after = int(np.max([((ltt - end_unix_time) // 3600) + 1 for key in hashtags]))


In [8]:
# Organize data into specific time frames.

# Initialize dictionary for each time frame.
data_hashtag_before = {}
data_hashtag_during = {}
data_hashtag_after = {}

# Iterate through each hashtag.
for key in hashtags:
    print(key)
    
    # Rename the dictionary value for readability
    temp_before = hashtag_dict_before[key]
    temp_during = hashtag_dict_during[key]
    temp_after = hashtag_dict_after[key]
    
    data_hashtag_before[key] = np.zeros((num_windows_before, 5)) # Initialize array: rows = time window, columns = feature
    num_followers_before = {} # Initialize dictionary to count # of followers for each tweet
    
    
    
    # Iterate through all elements before start time
    for i in range(np.shape(temp_before)[0]):
        # Get row number
        item_before = int(num_windows_before - 1 - ((start_unix_time - temp_before[i,0] - 1) // 3600))
        # Update first 3 elements (# of tweets, total # retweets, total # followers)
        data_hashtag_before[key][item_before] += np.array([1, int(temp_before[i, 1]), int(temp_before[i, 2]), 0, 0])
        # Get time of day (hour)
        dt_obj_pst = datetime.fromtimestamp(temp_before[i,0], pst_tz)
        data_hashtag_before[key][item_before][4] = int(datetime.strftime(dt_obj_pst, '%H'))
        # Get number of followers
        if item_before not in num_followers_before.keys():
            num_followers_before[item_before] = []
        num_followers_before[item_before].append(temp_before[i,2])
    for i in num_followers_before.keys():
        data_hashtag_before[key][i][3] = np.max(num_followers_before[i])
        
        
    # Iterate through all elements during time
    data_hashtag_during[key] = np.zeros((num_windows_during, 5))
    num_followers_during = {}
    for i in range(np.shape(temp_during)[0]):
        item_during = int(((temp_during[i,0] - start_unix_time) * 12) // 3600)
        data_hashtag_during[key][item_during] += np.array([1, int(temp_during[i, 1]), int(temp_during[i, 2]), 0, 0])
        dt_obj_pst = datetime.fromtimestamp(temp_during[i,0], pst_tz)
        data_hashtag_during[key][item_during][4] = int(datetime.strftime(dt_obj_pst, '%H'))
        
        if item_during not in num_followers_during.keys():
            num_followers_during[item_during] = []
        num_followers_during[item_during].append(temp_during[i,2])
    for i in num_followers_during.keys():
        data_hashtag_during[key][i][3] = np.max(num_followers_during[i])
        
    # Iterate through all elements after end time
    data_hashtag_after[key] = np.zeros((num_windows_after, 5))
    num_followers_after = {}
    for i in range(np.shape(temp_after)[0]):
        item_after = int((temp_after[i,0] - end_unix_time) // 3600)
        data_hashtag_after[key][item_after] += np.array([1, int(temp_after[i, 1]), int(temp_after[i, 2]), 0, 0])
        dt_obj_pst = datetime.fromtimestamp(temp_after[i,0], pst_tz)
        data_hashtag_after[key][item_after][4] = int(datetime.strftime(dt_obj_pst, '%H'))
        
        if item_after not in num_followers_after.keys():
            num_followers_after[item_after] = []
        num_followers_after[item_after].append(temp_after[i,2])
    for i in num_followers_after.keys():
        data_hashtag_after[key][i][3] = np.max(num_followers_after[i])
        
print('done')

gohawks
gopatriots
nfl
patriots
sb49
superbowl
done


In [9]:
# Aggregate data

# Initialize aggregated data variables
data_aggregate_before = np.zeros([num_windows_before, 5])
data_aggregate_during = np.zeros([num_windows_during, 5])
data_aggregate_after = np.zeros([num_windows_after, 5])

# Sum the # of tweets, total # of retweets, and # of followers
for key in hashtags:
    data_aggregate_before[:,0:3] += data_hashtag_before[key][:,0:3]
    data_aggregate_during[:,0:3] += data_hashtag_during[key][:,0:3]
    data_aggregate_after[:,0:3] += data_hashtag_after[key][:,0:3]
# Find the max # of followers for each
data_aggregate_before[:,3] = np.amax([data_hashtag_before[key][:,3] for key in hashtags], axis=0)
data_aggregate_during[:,3] = np.amax([data_hashtag_during[key][:,3] for key in hashtags], axis=0)
data_aggregate_after[:,3] = np.amax([data_hashtag_after[key][:,3] for key in hashtags], axis=0)

# Copy over the same time frames
data_aggregate_before[:,4] = data_hashtag_before['superbowl'][:,4]
data_aggregate_during[:,4] = data_hashtag_during['superbowl'][:,4]
data_aggregate_after[:,4] = data_hashtag_after['superbowl'][:,4]

### Nonlinear Regressions: Ensemble methods

#### Question 8

In [15]:
# Perform GridSearch
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import KFold
import time

param_grid = {
    'max_depth': [10, 20, 40, 60, 80, 100, 200, None],
    'max_features': ['auto', 'sqrt'],
    'min_samples_leaf': [1, 2, 4],
    'min_samples_split': [2, 5, 10],
    'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]
}

In [11]:
def show_results(grid_search_cv):
    print("Grid search scores on the basis of mean validation accuracy: ")
    print()
    means = grid_search_cv.cv_results_['mean_test_score']
    stds = grid_search_cv.cv_results_['std_test_score']
    params = grid_search_cv.cv_results_['params']
    for mean, std, param in zip(means, stds, params):
        print("%0.6f (+/-%0.06f) for %r"
              % (mean, std * 2, param))
    print()

    print("Optimal value of C: ")
    print()
    print(grid_search_cv.best_params_)
    print()

##### Analysis of data aggregated before Feb 1, 8:00 am

In [12]:
# Define train data and targets for BEFORE period
y_before = data_aggregate_before[1:,0] # Number of tweets (except first)
X_before = np.delete(data_aggregate_before, -1, 0) # Delete last row

print('X shape:', X_before.shape)
print('y shape:', y_before.shape)

X shape: (439, 5)
y shape: (439,)


In [ ]:
# Random Forest regressor grid search
print('performing RF grid search...')

start_time = time.time()
rf_cv_bef = GridSearchCV(RandomForestRegressor(), param_grid, cv=KFold(5, shuffle=True), scoring='neg_mean_squared_error',
                         verbose=1)
rf_cv_bef.fit(X_before, y_before)
print('grid search took {:.2f} seconds'.format(time.time() - start_time))

performing RF grid search...
Fitting 5 folds for each of 1440 candidates, totalling 7200 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


In [ ]:
show_results(rf_cv_bef)

In [ ]:
# GradientBoostingRegressor regressor grid search
print('performing GB grid search...')

start_time = time.time()
gb_cv_bef = GridSearchCV(GradientBoostingRegressor(), param_grid, cv=KFold(5, shuffle=True), scoring='neg_mean_squared_error',
                         verbose=1)
gb_cv_bef.fit(X_before, y_before)
print('grid search took {:.2f} seconds'.format(time.time() - start_time))

In [ ]:
show_results(gb_cv_bef)

##### Analysis of data aggregated between Feb 1, 8:00 am and Feb 1, 8:00 pm

In [ ]:
y_during = data_aggregate_during[1:,0]
X_during = np.delete(data_aggregate_during, -1, 0)

print('X shape:', X_during.shape)
print('y shape:', y_during.shape)

In [ ]:
# Random Forest regressor grid search
print('performing RF grid search...')

start_time = time.time()
rf_cv_dur = GridSearchCV(RandomForestRegressor(), param_grid, cv=KFold(5, shuffle=True), scoring='neg_mean_squared_error',
                         verbose=1)
rf_cv_dur.fit(X_during, y_during)
print('grid search took {:.2f} seconds'.format(time.time() - start_time))

In [ ]:
show_results(rf_cv_dur)

In [ ]:
# GradientBoostingRegressor regressor grid search
print('performing GB grid search...')

start_time = time.time()
gb_cv_dur = GridSearchCV(GradientBoostingRegressor(), param_grid, cv=KFold(5, shuffle=True), scoring='neg_mean_squared_error',
                         verbose=1)
gb_cv_dur.fit(X_during, y_during)
print('grid search took {:.2f} seconds'.format(time.time() - start_time))

In [ ]:
show_results(gb_cv_dur)

##### Analysis of data aggregated after Feb 1, 8:00 pm

In [ ]:
y_after = data_aggregate_after[1:,0]
X_after = np.delete(data_aggregate_after, -1, 0)

print('X shape:', X_after.shape)
print('y shape:', y_after.shape)

In [ ]:
# Random Forest regressor grid search
print('performing RF grid search...')

start_time = time.time()
rf_cv_aft = GridSearchCV(RandomForestRegressor(), param_grid, cv=KFold(5, shuffle=True), scoring='neg_mean_squared_error',
                         verbose=1)
rf_cv_aft.fit(X_after, y_after)
print('grid search took {:.2f} seconds'.format(time.time() - start_time))

In [ ]:
show_results(rf_cv_aft)

In [ ]:
# GradientBoostingRegressor regressor grid search
print('performing GB grid search...')

start_time = time.time()
gb_cv_aft = GridSearchCV(GradientBoostingRegressor(), param_grid, cv=KFold(5, shuffle=True), scoring='neg_mean_squared_error',
                         verbose=1)
gb_cv_aft.fit(X_after, y_after)
print('grid search took {:.2f} seconds'.format(time.time() - start_time))

In [ ]:
show_results(gb_cv_aft)